## Plots of SST for S-MODE region

... but using data directly from data providers, not the subsetted version 

To do this, I will try using the PO.DAAC Earthdata Cloud, hosted in Amazon Web Services (AWS).
To do that, I will try using the Harmony subsetting tool, based on tutorial here:
https://github.com/podaac/tutorials/tree/master/notebooks/harmony%20subsetting

In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc
import json
import requests
import sys
import shutil
import xarray as xa



def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


# GET TOKEN FROM CMR 
def get_token( url: str,client_id: str, user_ip: str,endpoint: str) -> str:
    try:
        token: str = ''
        username, _, password = netrc.netrc().authenticators(endpoint)
        xml: str = """<?xml version='1.0' encoding='utf-8'?>
        <token><username>{}</username><password>{}</password><client_id>{}</client_id>
        <user_ip_address>{}</user_ip_address></token>""".format(username, password, client_id, user_ip)
        headers: Dict = {'Content-Type': 'application/xml','Accept': 'application/json'}
        resp = requests.post(url, headers=headers, data=xml)
        
        response_content: Dict = json.loads(resp.content)
        token = response_content['token']['id']
    except:
        print("Error getting the token - check user name and password", sys.exc_info()[0])
    return token

In [2]:
collection = 'C1940473819-POCLOUD'
variable = 'sea_surface_temperature'
lat_var = 'lat'
lon_var = 'lon'
venue = 'prod'

In [3]:
cmr_root = 'cmr.earthdata.nasa.gov'
harmony_root = 'https://harmony.earthdata.nasa.gov'
edl_root = 'urs.earthdata.nasa.gov'

In [8]:
if venue == 'uat':
    cmr_root = 'cmr.uat.earthdata.nasa.gov'
    harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
    edl_root = 'uat.urs.earthdata.nasa.gov'

print ("Environments: ")
print ("\t" + cmr_root)
print ("\t" + harmony_root)
print ("\t" + edl_root)

Environments: 
	cmr.earthdata.nasa.gov
	https://harmony.earthdata.nasa.gov
	urs.earthdata.nasa.gov


In [20]:
setup_earthdata_login_auth(edl_root)
token_url="https://"+cmr_root+"/legacy-services/rest/tokens"
token='' # get_token(token_url,'jupyter', '127.0.0.1',edl_root)

Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to urs.earthdata.nasa.gov and will not be exposed in Jupyter
Username:jfarrar
········


In [21]:
cmr_url = "https://"+cmr_root+"/search/granules.umm_json?collection_concept_id="+collection+"&bounding_box=-90,-45.75,90,-45&token="+token
response = requests.get(cmr_url)

gid=response.json()['items'][0]['meta']['concept-id']
print(response.json()['items'][0])
print(gid)


{'meta': {'concept-type': 'granule', 'concept-id': 'G1966151265-POCLOUD', 'revision-id': 1, 'native-id': '20020704002506-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0', 'provider-id': 'POCLOUD', 'format': 'application/vnd.nasa.cmr.umm+json', 'revision-date': '2020-11-12T12:09:42.586Z'}, 'umm': {'TemporalExtent': {'RangeDateTime': {'EndingDateTime': '2002-07-04T00:29:59.000Z', 'BeginningDateTime': '2002-07-04T00:25:01.000Z'}}, 'MetadataSpecification': {'URL': 'https://cdn.earthdata.nasa.gov/umm/granule/v1.6.3', 'Name': 'UMM-G', 'Version': '1.6.3'}, 'GranuleUR': '20020704002506-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0', 'ProviderDates': [{'Type': 'Insert', 'Date': '2020-11-12T12:09:21.073Z'}, {'Type': 'Update', 'Date': '2020-11-12T12:09:21.073Z'}], 'SpatialExtent': {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -17.698, 'SouthBoundingCoordinate': -62.406, 'EastBoundingCoordinate': 24.515, 'NorthBoundingCoordinate': -40.386}]}}}, 'DataGran

In [22]:
bboxSubsetConfig = {
    'collection_id': collection,
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': gid,
    'lat': '(-45.75:45)',
    'lon': '(-90:90)'
}
bbox_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?granuleid={granuleid}&subset=lat{lat}&subset=lon{lon}'.format(**bboxSubsetConfig)
print('Request URL', bbox_url)


Request URL https://harmony.earthdata.nasa.gov/C1940473819-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleid=G1966151265-POCLOUD&subset=lat(-45.75:45)&subset=lon(-90:90)


In [23]:
with request.urlopen(bbox_url) as response, open('ogc_temp.nc', 'wb') as out_file:
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to ogc_temp.nc")

Content Size: 2844547
Downloaded request to ogc_temp.nc


In [24]:
ds = xa.open_dataset('ogc_temp.nc')
ds

<xarray.Dataset>
Dimensions:                      (ni: 1354, nj: 656, time: 1)
Coordinates:
    lat                          (nj, ni) float32 ...
    lon                          (nj, ni) float32 ...
  * time                         (time) datetime64[ns] 2002-07-04T00:25:01
Dimensions without coordinates: ni, nj
Data variables:
    sea_surface_temperature      (time, nj, ni) float32 ...
    sst_dtime                    (time, nj, ni) timedelta64[ns] ...
    quality_level                (time, nj, ni) float32 ...
    sses_bias                    (time, nj, ni) float32 ...
    sses_standard_deviation      (time, nj, ni) float32 ...
    l2p_flags                    (time, nj, ni) int16 ...
    sea_surface_temperature_4um  (time, nj, ni) float32 ...
    quality_level_4um            (time, nj, ni) float32 ...
    sses_bias_4um                (time, nj, ni) float32 ...
    sses_standard_deviation_4um  (time, nj, ni) float32 ...
    wind_speed                   (time, nj, ni) float32 ...
    dt_analysis                  (time, nj, ni) float32 ...
Attributes:
    Conventions:                CF-1.7, ACDD-1.3
    title:                      MODIS Aqua L2P SST
    summary:                    Sea surface temperature retrievals produced a...
    references:                 GHRSST Data Processing Specification v2r5
    institution:                NASA/JPL/OBPG/RSMAS
    history:                    MODIS L2P created at JPL PO.DAAC\n2021-10-13 ...
    comment:                    L2P Core without DT analysis or other ancilla...
    license:                    GHRSST and PO.DAAC protocol allow data use as...
    id:                         MODIS_A-JPL-L2P-v2019.0
    naming_authority:           org.ghrsst
    product_version:            2019.0
    uuid:                       f6e1f61d-c4a4-4c17-8354-0c15e12d688b
    gds_version_id:             2.0
    netcdf_version_id:          4.1
    date_created:               20200727T162520Z
    file_quality_level:         3
    spatial_resolution:         1km
    start_time:                 20020704T002501Z
    time_coverage_start:        20020704T002501Z
    stop_time:                  20020704T002959Z
    time_coverage_end:          20020704T002959Z
    northernmost_latitude:      -40.3864
    southernmost_latitude:      -62.4058
    easternmost_longitude:      24.5151
    westernmost_longitude:      -17.6983
    source:                     MODIS sea surface temperature observations fo...
    platform:                   Aqua
    sensor:                     MODIS
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.01
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.01
    acknowledgment:             The MODIS L2P sea surface temperature data ar...
    creator_name:               Ed Armstrong, JPL PO.DAAC
    creator_email:              edward.m.armstrong@jpl.nasa.gov
    creator_url:                http://podaac.jpl.nasa.gov
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L2P
    cdm_data_type:              swath
    startDirection:             Descending
    endDirection:               Descending
    day_night_flag:             Night
    history_json:               [{"date_time": "2021-10-13T22:50:30.055500+00...

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime

import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw

# For great circle distance between two points, use gsw.geostrophy.distance(lon, lat, p=0, axis=-1)


In [2]:
# %matplotlib qt 
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + 'SMODE_'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-126,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,11)
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 40)
    levels = np.linspace(13,18,11)

    

In [4]:
def plot_ops_area(ax,**kwargs):
    """ Add polygon to show S-MODE pilot operations area
         Inputs
            - matplotlib.pyplot.plot kwargs

         return
            - exit code (True if OK)

    """

    # Add S-MODE pilot operations area
    ''' 
    Old corners of pentagon:
    38° 05.500’ N, 125° 22.067’ W
    37° 43.000’ N, 124° 00.067’ W
    37° 46.417’ N, 123° 10.000‘ W
    37° 05.000’ N, 122° 43.067’ W
    36° 20.000’ N, 124° 19.067’ W 
    
    New points:
    37° 45.000’ N, 123° 26.000‘ W
    replaces
    37° 46.417’ N, 123° 10.000‘ W
    --and--
    36° 58.000’ N, 122° 57.000’ W
    replaces
    37° 05.000’ N, 122° 43.067’ W

    New corners of pentagon:
    38° 05.500’ N, 125° 22.067’ W
    37° 43.000’ N, 124° 00.067’ W
    37° 45.000’ N, 123° 26.000‘ W
    36° 58.000’ N, 122° 57.000’ W
    36° 20.000’ N, 124° 19.067’ W 
    '''
    
    coord = [[-(125+22.067/60),38+5.5/60], [-(124+0.067/60),37+43/60], [-(123+26/60),37+45/60], [-(122+57/60),36+58/60], [-(124+19.067/60),36+20/60]]
    coord.append(coord[0]) #repeat the first point to create a 'closed loop'

    xs, ys = zip(*coord) #create lists of x and y values

    if ax is None:
        ax = plt.gca()    
    # ax.plot(xs,ys,transform=ccrs.PlateCarree()) 
    ax.plot(xs,ys,**kwargs) 
    
    SF_lon=-(122+25/60)
    SF_lat= 37+47/60
    
    # mark a known place to help us geo-locate ourselves
    ax.plot(SF_lon, SF_lat, 'o', markersize=3, **kwargs)
    ax.text(SF_lon-5/60, SF_lat+5/60, 'San Francisco', fontsize=8, **kwargs)
    ax.text(np.mean(xs)-.6, np.mean(ys)-.3, 'S-MODE ops area', fontsize=8, **kwargs)
    print(kwargs)
    
    return(xs,ys,ax)


In [ ]:
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPB/AVHRR_METOPB_20210930T060000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20210930T065001Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc'  #This one is good

url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20211011T233459Z.nc'
# http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc
ds = xr.open_dataset(url)


In [ ]:
#####################################
# This is AVHRR from VIIRS NPP
# This takes a long time-- is there a faster way? 
url = 'https://thredds.jpl.nasa.gov/thredds/dodsC/OceanTemperature/VIIRS_NPP-OSPO-L3U-v2.61.nc'
ds_npp = xr.open_dataset(url)
#ds_sub = ds_npp.sea_surface_temperature.sel(time=slice('20210929','20220101'),lat=slice(ymax,ymin),lon=slice(xmin,xmax),drop=True)-273.15

ff = ~np.isnan(ds_sub.sel(lat=slice(37.55, 37.45), lon=slice(-124.05, -124.95)).mean('lon').mean('lat'))
ff2 = np.where(ff)
fig = plt.figure(figsize=(8, 4))
plt.plot(ff)
plt.title('Indices of subset with non-nan data')



In [6]:
#####################################
# VIIRS another way
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L2P/VIIRS_NPP/NAVO/v3/2021/286/20211013000106-NAVO-L2P_GHRSST-SST1m-VIIRS_NPP-v02.0-fv03.0.nc'
ds = xr.open_dataset(url)
# ds_sub = ds.sea_surface_temperature.sel(time=slice('20210929','20220101'),lat=slice(ymax,ymin),lon=slice(xmin,xmax),drop=True)-273.15


In [7]:
ds

<xarray.Dataset>
Dimensions:                      (ni: 3200, nj: 768, time: 1)
Coordinates:
    lat                          (nj, ni) float32 ...
    lon                          (nj, ni) float32 ...
  * time                         (time) datetime64[ns] 2021-10-13T00:01:06
Dimensions without coordinates: ni, nj
Data variables:
    sea_surface_temperature      (time, nj, ni) float32 ...
    sst_dtime                    (time, nj, ni) float32 ...
    sses_bias                    (time, nj, ni) float32 ...
    sses_standard_deviation      (time, nj, ni) float32 ...
    dt_analysis                  (time, nj, ni) float32 ...
    wind_speed                   (time, nj, ni) float32 ...
    aerosol_dynamic_indicator    (time, nj, ni) float32 ...
    adi_dtime_from_sst           (time, nj, ni) float32 ...
    satellite_zenith_angle       (time, nj, ni) float32 ...
    l2p_flags                    (time, nj, ni) float32 ...
    quality_level                (time, nj, ni) float32 ...
    brightness_temperature_4um   (time, nj, ni) float32 ...
    brightness_temperature_11um  (time, nj, ni) float32 ...
    brightness_temperature_12um  (time, nj, ni) float32 ...
Attributes:
    Conventions:                CF-1.6
    title:                      VIIRS L2P SST
    summary:                    Sea Surface temperature retrievals produced a...
    references:                 NAVOCEANO MCSST
    institution:                NAVO
    history:                    Created with VSST on  2021/10/13 at 01:05:27 UT
    comment:                    Improved cloud screening with emphasis on the...
    license:                    GHRSST protocol describes data use as free an...
    id:                         NAVO-L2P-VIIRS
    naming_authority:           org.ghrsst
    product_version:            03.0
    uuid:                       3c27d404-33ef-4096-899b-ece1f969f774
    gds_version_id:             02.0
    netcdf_version_id:          4.3.3.1 of Dec 10 2015 16:44:18 $
    date_created:               20211013T010527
    file_quality_level:         3
    spatial_resolution:         1500 m at nadir
    start_time:                 20211013T000106Z
    time_coverage_start:        20211013T000106Z
    stop_time:                  20211013T000231Z
    time_coverage_end:          20211013T000231Z
    northernmost_latitude:      14.850611
    southernmost_latitude:      5.6697326
    easternmost_longitude:      -146.35007
    westernmost_longitude:      -175.09744
    source:                     GMODO_npp,SVM05_npp,SVM07_npp,SVM012_npp,SVM1...
    platform:                   NPP
    sensor:                     VIIRS
    metadata_link:              http://podaac.jpl.nasa.gov:8890/ws/metadata/d...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.0075
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.0075
    acknowledgment:             The data from the Naval Oceanographic Office ...
    creator_name:               NAVOCEANO MCSST GROUP
    creator_email:              navo_stns_mcsst@navy.mil
    creator_url:                http://www.usno.navy.mil/NAVO
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L2P
    cdm_data_type:              swath

In [ ]:
ff = ~np.isnan(ds_sub.sel(lat=slice(37.55, 37.45), lon=slice(-124.05, -124.95)).mean('lon').mean('lat'))
ff2 = np.where(ff)
fig = plt.figure(figsize=(8, 4))
plt.plot(ff)
plt.title('Indices of subset with non-nan data')